# Lab: Use Image Generator

Learn to use Image Generator

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elephantscale/cool-ML-demos/blob/main/image/img2-image-generator.ipynb)

### Runtime
~20 minutes

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

In [ ]:
## Loading our custom utils files
import os
import sys
from pathlib import Path

# Hack to download image utils when running on Colab ..etc
import os
import urllib.request

file_url = 'https://raw.githubusercontent.com/elephantscale/es-public/master/deep-learning/image_utils.py'
file_location = "image_utils.py"

if not os.path.exists (file_location):
    file_location = os.path.basename(file_location)
    if not os.path.exists(file_location):
        print("Downloading : ", file_url)
        urllib.request.urlretrieve(file_url, file_location)
# print('file_location:', file_location)

## Step 1: Download Data

In [ ]:
import os

data_location = 'https://elephantscale-public.s3.amazonaws.com/data/images/cat-dog-redux.zip'

data_location_local = keras.utils.get_file(fname=os.path.basename(data_location),
                                           origin=data_location, extract=True)
print ('local download file: ', data_location_local)
data_dir = os.path.join(os.path.dirname(data_location_local), 'cat-dog-redux')
print ('local data dir: ', data_dir)
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'val')
print ('train dir:', train_dir)
print ('validation dir:', validation_dir)

In [ ]:
from image_utils import print_training_validation_stats

print_training_validation_stats(train_dir, validation_dir)

In [ ]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Here we are rescaling images by dividing it by 255
## We are shuffling the images to increase randomness
## Images are reshaped to 150x150
## Since we have two images, the class is binary

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
# we can get images per batch

sample_training_images, sample_training_labels = next(train_data_gen)
sample_validation_images, sample_validation_labels = next(val_data_gen)

print ('# sample training images: ', len(sample_training_images))
print ('# sample validation images: ', len(sample_validation_images))
print ('sample training labels : ', sample_training_labels[:10])

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    

print ("training images...")
plotImages(sample_training_images)

print ("validation images...")
plotImages(sample_validation_images)

## More Image Generator Attributes

In [ ]:
## directory
print ('image dir :', train_data_gen.directory)
print()

## Image count
print ("total training images : ", train_data_gen.n)
print()

## Class indexes 
print ("Class indices : ", train_data_gen.class_indices)
print ()

## Some file names
print ('file names : ', train_data_gen.filenames[:5])
print ()

## file paths
print ('file paths : ', train_data_gen.filepaths[:5])
print ()

## Augmenting Images

We augment images
- to produce more variants of training images (so the algorithm gets more resilient)
- increase training data size
- to prevent overfitting

### Horizontal Flip

In [ ]:
## Let's start with horizontal flip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator2 = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('horizontal flip 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('horizontal flip 2')
plotImages (augmented_images)

### Rotatation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator2 = ImageDataGenerator(rescale=1./255, rotation_range=45)

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('rotated images 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('rotated images 2')
plotImages (augmented_images)

### Zoom

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# zoom_range from 0 - 1 where 1 = 100%.
train_image_generator2 = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('zoomed images 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('zoomed images 2')
plotImages (augmented_images)

## Shearing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# shear_range = 0 to 1
train_image_generator2 = ImageDataGenerator(rescale=1./255, shear_range=0.3)

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('sheard images 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('sheard images 2')
plotImages (augmented_images)

## Brightness

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Values over 1.0 will brighten the image; values less than 1.0 will darken the image;  1.0 no change
train_image_generator2 = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,0.7])

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('brightness images 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('brightness images 2')
plotImages (augmented_images)

### Putting it all together

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator2 = ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.15,
                            height_shift_range=.15,
                            horizontal_flip=True,
                            zoom_range=0.5,
                            #brightness_range=[0.2,0.7],
                            shear_range=0.3,
                            fill_mode='nearest'
                            )

train_data_gen2 = train_image_generator2.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen2[0][0][0] for i in range(5)]
print ('augmented images 1')
plotImages (augmented_images)

augmented_images = [train_data_gen2[0][0][1] for i in range(5)]
print ('augmented images 2')
plotImages (augmented_images)